In [1]:
%pip install xmltodict

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import xmltodict
import json

# Read XML data
path = 'github/server/data/resid/'
with open(path + 'RESIDUES.xml', 'r', encoding='utf-8') as xml_file:
    xml_data = xml_file.read()

# Parse XML to Python dictionary
data_dict = xmltodict.parse(xml_data)

# Convert dictionary to JSON
json_data = json.dumps(data_dict, indent=4)

# Optionally, save to a file
with open(path + 'RESIDUES.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_data)

In [ ]:
import requests
import re
from collections import OrderedDict
from typing import Literal

def get_jpred_inference(seq: str, res_type: Literal['simple', 'full']) -> str:
    sequence_query = ">query\n{}".format(seq)

    parameters_dict = OrderedDict([("skipPDB", True),
                                   ("format", 'seq')])

    parameters_list = ["{}={}".format(k, v) for k, v in parameters_dict.items() if v]
    parameters_list.append(sequence_query)
    query = u"£€£€".join(parameters_list)

    response = requests.post(
        'http://www.compbio.dundee.ac.uk/jpred4/cgi-bin/rest/job',
        data=query.encode('utf-8'),
        headers={"Content-type": "text/txt"}
    )

    if response.status_code == 202:
        result_url = response.headers['Location']
        jobid = re.search(r"(jp_.*)$", result_url).group(1)

        response = requests.Response()
        while 'finished' not in response.text.lower():
            print("Trying...")
            response = requests.get(f'http://www.compbio.dundee.ac.uk/jpred4/cgi-bin/rest/job/id/{jobid}')
        if res_type == 'simple':
            results_url = response.text.split('Results available at the following URL:')[-1].strip().split('.results')[0] + '.svg.html'
        elif res_type == 'full':
            results_url = response.text.split('Results available at the following URL:')[-1].strip().split('.results')[0] + '.html'

        response = requests.get(results_url)
        return response.text # HTML here

    else:
        return "Could not submit job!"

In [210]:
sequence = "MNPELQSAIGQGAALKHAETVDKSAPQIENVTVKKVDRSSFLEEVAKPHELKHAETVDKSGPAIPEDVHVKKVDRGAFLSEIEKAAKQ"

In [211]:
response = get_jpred_inference(sequence, 'full')

skipPDB=True£€£€format=seq£€£€>query
MNPELQSAIGQGAALKHAETVDKSAPQIENVTVKKVDRSSFLEEVAKPHELKHAETVDKSGPAIPEDVHVKKVDRGAFLSEIEKAAKQ


In [159]:
response

'<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 3.2//EN"><html><head><meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1"><title>Jpred secondary structure prediction results</title><script defer data-domain="compbio.dundee.ac.uk" src="https://analytics.compbio.dundee.ac.uk/js/script.js"></script></head><body bgcolor="#ffffff"><pre><code>QUERY                  : MNPELQSAIGQGAALKHAETVDKSAPQIENVTVKKVDRSSFLEEVAKPHELKHAETVDKSGPAIPEDVHVKKVDRGAFLSEIEKAAKQ : QUERY \n<a href=http://www.ebi.ac.uk/ebisearch/search.ebi?db=allebi&query=UniRef90_W4WHP5>UniRef90_W4WHP5</a>        : --ASMMKEVESGKTLKHVKCNDRSAPLIERVSEKSNMHNQLLKQIQQGVKLKTVKTNDRSKPILEG---LRKFRRQMTIEEQIQKAEE : <a href=http://www.ebi.ac.uk/ebisearch/search.ebi?db=allebi&query=UniRef90_W4WHP5>UniRef90_W4WHP5</a>\n<a href=http://www.ebi.ac.uk/ebisearch/search.ebi?db=allebi&query=UniRef90_W4WHP5>UniRef90_W4WHP5</a>        : --------------------------ILKSRPRKRPDWASMMKEVESGKTLKHVKCNDRSAPLIERVNKVKSNMHNQLLKQIQQGVK- : <a href=http://www

In [167]:
from bs4 import BeautifulSoup

def format_response(response: str) -> dict[str, dict[str, str]]:
    parsed_html = BeautifulSoup(response)
    code_chunk: str = parsed_html.find('code').text
    response = {'alignment': {}, 'prediction': {}}

    for line in code_chunk.split('\n'):
        key, value = '', ''
        if line != '':
            line = line.split(':')
            key, value = line[0].strip(), line[1].strip()
            if 'UniRef' in key or 'QUERY' in key:
                response['alignment'][key] = value
            else:
                if key == '':
                    key = 'Pos'
                response['prediction'][key] = value
    return response

In [168]:
sequences = format_response(response)

In [169]:
sequences

{'alignment': {'QUERY': 'MNPELQSAIGQGAALKHAETVDKSAPQIENVTVKKVDRSSFLEEVAKPHELKHAETVDKSGPAIPEDVHVKKVDRGAFLSEIEKAAKQ',
  'UniRef90_W4WHP5': '--------------------------ILKSRPRKRPDWASMMKEVESGKTLKHVKCNDRSAPLIERVNKVKSNMHNQLLKQIQQGVK-',
  'UniRef90_R1DP50': '-----LTAIQKGTTLKPTQTNDRSEPVKESVTIKASPMNQLKEELKEKPKLNHVDTVDKSAPMIESSTKIAKDARPQLMSELLTKS--',
  'UniRef90_F0ZMH6': '--DDLLSDISKGANLNKTDTNDRSKPAIDNTHIKENNKGALLSEIK------------------------------------------',
  'UniRef90_Q55DU1': '--------------------------------------NPLLAEINKGTDLKHAETQDKSAPIIE-NVPIKKNDHSSLLGEVEKGAQ-',
  'UniRef90_UPI00022C8B2B': '--ASMMKEVESGKPLRHVKCNDRSAPLIERVSEKSNMHNQLLKQIQEGVKLKRVQTNDRSRPMLQG---LRKFRRQLTIEEQMQKSEE',
  'UniRef90_L8GHP1': '--ARLLNEVREGAQLKHVATEDKSKPTIDDTQLKTWDRQGFLDEVRIGTELRHI----------------------------------',
  'UniRef90_F4PK10': 'MSDALFSQVQAGANLKKAETVDKSGPVIDDVHVKENNHGALLKELSTDHKLNKAETVDKSGPVIEKDVHVKENNRGALLGEIKAKA--',
  'UniRef90_R1DJB3': '---ELMAEVAKGKSLNHVETDDKSAPAVEDVKIKPNPRVALLNDVCKKV---------------

In [205]:
def generate_html(data: dict[str, dict[str, str]]):
    # Calculate the maximum label length for the alignment section
    max_label_length = max(len(key) for key in data['alignment'].keys())
    # Set a base font size for both labels and sequences
    font_size = '14px'
    
    # Start the HTML structure
    html_content = f'''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Sequence Alignment and Prediction</title>
        <style>
            body {{
                font-family: 'Courier New', Courier, monospace;
                padding: 20px;
            }}
            h2 {{
                color: #4CAF50;
            }}
            .section {{
                margin-bottom: 20px;
            }}
            .alignment-container, .prediction-container {{
                display: flex;  /* Flexbox for horizontal layout */
                justify-content: space-between;
                align-items: flex-start;
            }}
            .labels-box, .sequences-box {{
                background-color: #f4f4f4;
                padding: 15px;
                border-radius: 5px;
                flex: 1;  /* Both boxes take up equal width */
                margin-right: 10px;  /* Space between the boxes */
                font-size: {font_size};
            }}
            .labels-box {{
                max-width: 200px;  /* Dynamically set the label width (10px per character and some padding) */
            }}
            .sequence-label {{
                font-weight: bold;
                margin-bottom: 10px;
                white-space: nowrap;  /* Prevent label text from wrapping */
            }}
            .sequence {{
                white-space: pre-wrap;  /* Sequence wraps to fit in the box */
                margin-bottom: 10px;
            }}
            pre {{
                background-color: #f4f4f4;
                margin: 0;
                padding: 0;
                white-space: pre-wrap;
                word-wrap: break-word;
            }}
        </style>
    </head>
    <body>
    '''.strip()

    # Add Alignment Data (Two boxes side by side)
    html_content += '<h2>Sequence Alignment Data</h2><div class="section alignment-container">'

    # Left box for labels
    html_content += '<div class="labels-box">'
    for key in data['alignment'].keys():
        html_content += f'''
        <div class="sequence-label" style="color="{'red' if 'QUERY' in key else 'black'}";">{key}</div>
        '''.strip()
    html_content += '</div>'  # Close labels box

    # Right box for sequences
    html_content += '<div class="sequences-box">'
    for sequence in data['alignment'].values():
        html_content += f'''
        <div class="sequence">{sequence}</div>
        '''.strip()
    html_content += '</div>'  # Close sequences box

    html_content += '</div>'  # Close alignment container

    # # Add Prediction Data (Two boxes side by side)
    html_content += '<h2>Prediction Data</h2><div class="section prediction-container">'

    # Left box for prediction labels (QUERY, Pos, OrigSeq, etc.)
    html_content += '<div class="labels-box">'
    for key in data['prediction'].keys():
        html_content += f'''
        <div class="sequence-label">{key}</div>
        '''.strip()
    html_content += '</div>'  # Close labels box

    # Right box for prediction sequences
    html_content += '<div class="sequences-box">'
    for prediction in data['prediction'].values():
        html_content += f'''
        <div class="sequence"><pre>{prediction}</pre></div>
        '''.strip()
    html_content += '</div>'  # Close sequences box

    html_content += '</div>'  # Close prediction container

    # Close the body and html tags
    html_content += '''
    </body>
    </html>
    '''

    return html_content

In [206]:

html_content = generate_html(sequences)
print(html_content)

<!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Sequence Alignment and Prediction</title>
        <style>
            body {
                font-family: 'Courier New', Courier, monospace;
                padding: 20px;
            }
            h2 {
                color: #4CAF50;
            }
            .section {
                margin-bottom: 20px;
            }
            .alignment-container, .prediction-container {
                display: flex;  /* Flexbox for horizontal layout */
                justify-content: space-between;
                align-items: flex-start;
            }
            .labels-box, .sequences-box {
                background-color: #f4f4f4;
                padding: 15px;
                border-radius: 5px;
                flex: 1;  /* Both boxes take up equal width */
                margin-right: 10px;  /* Space between the bo

In [212]:
"£€£€".encode('utf-8')

b'\xc2\xa3\xe2\x82\xac\xc2\xa3\xe2\x82\xac'

In [1]:
# from PIL import Image
# from glob import glob

# files = [i for i in glob('github/server/data/resid/images/*.GIF')]
# print(files)
# for file in files:
#     im = Image.open(file)
#     im.convert("RGBA")
#     im.save(file.split('.')[0] + '.png')

In [40]:
import requests
from io import BytesIO;

acc = 'P04637'

response = requests.get(f"https://alphafold.ebi.ac.uk/api/prediction/{acc}")
with open('temp.pdb', 'w') as f:
    byte_file = BytesIO(requests.get(response.json()[0]['pdbUrl']).text.encode('utf-8'))


In [42]:
import mdtraj as md

In [43]:
# Load the PDB structure
traj = md.load(byte_file.decode('utf-8'))

# Compute the solvent-accessible surface area
sasa = md.shrake_rupley(traj)

# Print the SASA for each atom
for i, value in enumerate(sasa[0]):  # Iterate over atoms in the first frame
    print(f"Atom {i}: SASA = {value:.16f} Å²")

AttributeError: '_io.BytesIO' object has no attribute 'decode'

In [12]:
for i, value in enumerate(md.compute_dssp(traj)[0]):
    print(f"Atom {i+1}: {value}")

Atom 1: C
Atom 2: C
Atom 3: C
Atom 4: C
Atom 5: C
Atom 6: C
Atom 7: C
Atom 8: H
Atom 9: H
Atom 10: H
Atom 11: H
Atom 12: H
Atom 13: H
Atom 14: H
Atom 15: C
Atom 16: C
Atom 17: C
Atom 18: C
Atom 19: C
Atom 20: E
Atom 21: E
Atom 22: E
Atom 23: E
Atom 24: H
Atom 25: H
Atom 26: H
Atom 27: H
Atom 28: H
Atom 29: C
Atom 30: C
Atom 31: C
Atom 32: C
Atom 33: C
Atom 34: C
Atom 35: C
Atom 36: C
Atom 37: C
Atom 38: C
Atom 39: C
Atom 40: C
Atom 41: C
Atom 42: C
Atom 43: C
Atom 44: C
Atom 45: H
Atom 46: H
Atom 47: H
Atom 48: H
Atom 49: H
Atom 50: H
Atom 51: H
Atom 52: H
Atom 53: H
Atom 54: E
Atom 55: E
Atom 56: E
Atom 57: E
Atom 58: E
Atom 59: C
Atom 60: C
Atom 61: C
Atom 62: C
Atom 63: C
Atom 64: C
Atom 65: C
Atom 66: C
Atom 67: C
Atom 68: C
Atom 69: C
Atom 70: C
Atom 71: C
Atom 72: C
Atom 73: C
Atom 74: C
Atom 75: C
Atom 76: C
Atom 77: H
Atom 78: H
Atom 79: H
Atom 80: H
Atom 81: H
Atom 82: H
Atom 83: H
Atom 84: H
Atom 85: H
Atom 86: H
Atom 87: H
Atom 88: H
Atom 89: H
Atom 90: H
Atom 91: C
Atom 92:

In [4]:
for i, value in enumerate(md.compute_dssp(traj, simplified=False)[0]):
    print(f"Atom {i+1}: {value}")

NameError: name 'traj' is not defined

In [15]:

traj = md.load("7bwn.pdb")
print(traj.topology)

<mdtraj.Topology with 32 chains, 3062 residues, 20596 atoms, 20437 bonds>


In [39]:
top: md.Topology = traj.topology

from mdtraj.core.topology import Chain

for r in top.chains:
    r: Chain = r
    for i in range(r.n_residues):
        print(r.residue(i), end='')
    print()

MET1SER2LYS3GLY4GLU5GLU6LEU7PHE8THR9GLY10VAL11VAL12PRO13ILE14LEU15VAL16GLU17LEU18ASP19GLY20ASP21VAL22ASN23GLY24HIS25LYS26PHE27SER28VAL29ARG30GLY31GLU32GLY33GLU34GLY35ASP36ALA37THR38ASN39GLY40LYS41LEU42THR43LEU44LYS45PHE46ILE47CYS48THR49THR50GLY51LYS52LEU53PRO54VAL55PRO56TRP57PRO58THR59LEU60VAL61THR62THR63LEU64THR65TYR66GLY67VAL68GLN69CYS70PHE71SER72ARG73TYR74PRO75ASP76HIS77MET78LYS79ARG80HIS81ASP82PHE83PHE84LYS85SER86ALA87MET88PRO89GLU90GLY91TYR92VAL93GLN94GLU95ARG96THR97ILE98SER99PHE100LYS101ASP102ASP103GLY104THR105TYR106LYS107THR108ARG109ALA110GLU111VAL112LYS113PHE114GLU115GLY116ASP117THR118LEU119VAL120ASN121ARG122ILE123GLU124LEU125LYS126GLY127ILE128ASP129PHE130LYS131GLU132ASP133GLY134ASN135ILE136LEU137GLY138HIS139LYS140LEU141GLU142TYR143ASN144PHE145ASN146SER147HIS148ASN149VAL150TYR151ILE152THR153ALA154ASP155LYS156GLN157LYS158ASN159GLY160ILE161LYS162ALA163ASN164PHE165LYS166ILE167ARG168HIS169ASN170VAL171GLU172ASP173GLY174SER175VAL176GLN177LEU178ALA179ASP180HIS181TYR182GLN183GLN184ASN1

In [ ]:
li = [i for i in traj.topology.to_fasta() if i != '']
li = [r for i, r in enumerate(li) if i%2 == 0]

In [13]:
sasa = md.shrake_rupley(traj, mode='residue')[0]
for i, s in enumerate(sasa):
    print(traj.topology.residue(i), i+1, s)

MET1 1 0.99473655
SER2 2 0.16673219
LYS3 3 1.0721238
GLY4 4 0.015996926
GLU5 5 0.30889145
GLU6 6 1.3146064
LEU7 7 0.4000269
PHE8 8 0.042165518
THR9 9 1.0056454
GLY10 10 0.23649387
VAL11 11 0.82704616
VAL12 12 0.013837408
PRO13 13 0.60801435
ILE14 14 0.06644051
LEU15 15 0.28932762
VAL16 16 0.012579462
GLU17 17 0.4893865
LEU18 18 0.018210491
ASP19 19 0.7251508
GLY20 20 0.08032402
ASP21 21 0.32502833
VAL22 22 0.0012579461
ASN23 23 0.5495002
GLY24 24 0.35285854
HIS25 25 0.6617523
LYS26 26 1.0189892
PHE27 27 0.04814585
SER28 28 0.27696133
VAL29 29 0.042288717
ARG30 30 0.62781256
GLY31 31 0.0012579461
GLU32 32 0.87413967
GLY33 33 0.15013069
GLU34 34 0.6831556
GLY35 35 0.0037738383
ASP36 36 0.21188775
ALA37 37 0.023333605
THR38 38 0.78907645
ASN39 39 0.8089521
GLY40 40 0.020515386
LYS41 41 0.65661323
LEU42 42 0.057865523
THR43 43 0.411748
LEU44 44 0.026613636
LYS45 45 0.36763674
PHE46 46 0.0
ILE47 47 0.11069926
CYS48 48 0.011161031
THR49 49 0.62822014
THR50 50 0.7328234
GLY51 51 0.30722162
LY

In [2]:
import os
import tempfile
import requests
import mdtraj as md
from mdtraj.core.topology import Topology, Chain
from mdtraj.core.trajectory import Trajectory

In [12]:

acc = 'P04637'
_id = '1A1U'

# response = requests.get(requests.get(f"https://alphafold.ebi.ac.uk/api/prediction/{acc}").json()[0]['pdbUrl']).text
response = requests.get(f"https://files.rcsb.org/download/{_id}.pdb").text
file = tempfile.NamedTemporaryFile(suffix='.pdb', delete=False)
# sequence = info['uniprotSequence']
file.write(
    response.encode('utf-8')
)

traj: Trajectory = md.load(file.name)
# traj.atom_slice(atom_indices=traj.topology.select('chainid 0'), inplace=True)

# chains = [traj.top.chain(i).chain_id for i in range(1, traj.top.n_chains)]
# p = top.select('chainid 0').tolist()
# new_traj = traj.atom_slice()

# show = 'sasa'
for i in range(traj.n_chains):
    chain: Chain = traj.top.chain(i)
    print(chain.__dict__)

# for i in range(traj.n_chains):

#     show = 'sasa'

#     if show == 'sasa':
#         for i, value in enumerate(md.shrake_rupley(chain, mode='residue')[0]):  # Iterate over atoms in the first frame
#             print(f"For Residue {chain.residue(i)}: SASA = {value:.16f} Å²")
#     elif show == 'ss':
#         for i, value in enumerate(md.compute_dssp(chain, simplified=False)[0]):  # Iterate over atoms in the first frame
#             print(f"For Residue {chain.residue(i)}: SS = {value}")
#     break

file.close()
os.unlink(file.name)

{'index': 0, 'topology': <mdtraj.Topology with 2 chains, 58 residues, 1016 atoms, 1022 bonds at 0x18ad6c1d0c0>, '_residues': [GLU26, TYR27, PHE28, THR29, LEU30, GLN31, ILE32, ARG33, GLY34, ARG35, GLU36, ARG37, PHE38, GLU39, LYS40, ILE41, ARG42, GLU43, TYR44, ASN45, GLU46, ALA47, LEU48, GLU49, LEU50, LYS51, ASP52, ALA53, GLN54], 'chain_id': 'A'}
{'index': 1, 'topology': <mdtraj.Topology with 2 chains, 58 residues, 1016 atoms, 1022 bonds at 0x18ad6c1d0c0>, '_residues': [GLU26, TYR27, PHE28, THR29, LEU30, GLN31, ILE32, ARG33, GLY34, ARG35, GLU36, ARG37, PHE38, GLU39, LYS40, ILE41, ARG42, GLU43, TYR44, ASN45, GLU46, ALA47, LEU48, GLU49, LEU50, LYS51, ASP52, ALA53, GLN54], 'chain_id': 'C'}


In [50]:
import requests
subsequence = "ALALSFGSHECDASP"
ptm = 'Phosphorylation'

response = requests.post(
    'http://localhost:8000/ptmkb/api/calculate-propensity',
    json={
        'subsequence': subsequence,
        'ptm': ptm
    }
)
if response.ok:
    print(response.json())

{'additive_score': -40.583825223908555, 'multiplicative_score': 1917949.1046021783, 'adjusted_multiplicative_score': -14.466766998412432}


In [51]:
subsequence = 'WKLLPENNVLSPLPSQAMDDW'
len(subsequence)
subsequence[len(subsequence) // 2]

'S'

In [52]:
data = {
    "simplified": [
        "C",
        "C",
        "C",
        "C",
        "C",
        "C",
        "C",
        "C",
        "C",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "C"
    ],
    "detailed": [
        " ",
        " ",
        " ",
        " ",
        " ",
        " ",
        " ",
        " ",
        "S",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        "H",
        " "
    ],
    "SASA": [
        2.090446710586548,
        2.0861287117004395,
        1.9912043809890747,
        1.0552688837051392,
        1.4099764823913574,
        1.449099063873291,
        0.21670609712600708,
        2.0954129695892334,
        0.5228367447853088,
        2.2717714309692383,
        1.3273906707763672,
        1.5165772438049316,
        1.2684755325317383,
        0.8443407416343689,
        1.5230543613433838,
        0.32844826579093933,
        1.5948951244354248,
        1.150672435760498,
        1.1325087547302246,
        0.6986484527587891,
        1.3253200054168701,
        0.6202453970909119,
        1.059617519378662,
        1.0855765342712402,
        0.9763954877853394,
        1.6857918500900269,
        1.1609500646591187,
        0.6698938012123108,
        1.7914258241653442
    ],
    "sequence": "EYFTLQIRGRERFEKIREYNEALELKDAQ"
}

In [ ]:
import json
json.dumps(data)

'{"simplified": ["C", "C", "C", "C", "C", "C", "C", "C", "C", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "C"], "detailed": [" ", " ", " ", " ", " ", " ", " ", " ", "S", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", " "], "SASA": [2.090446710586548, 2.0861287117004395, 1.9912043809890747, 1.0552688837051392, 1.4099764823913574, 1.449099063873291, 0.21670609712600708, 2.0954129695892334, 0.5228367447853088, 2.2717714309692383, 1.3273906707763672, 1.5165772438049316, 1.2684755325317383, 0.8443407416343689, 1.5230543613433838, 0.32844826579093933, 1.5948951244354248, 1.150672435760498, 1.1325087547302246, 0.6986484527587891, 1.3253200054168701, 0.6202453970909119, 1.059617519378662, 1.0855765342712402, 0.9763954877853394, 1.6857918500900269, 1.1609500646591187, 0.6698938012123108, 1.7914258241653442], "sequence": "EYFTLQIRGRERFEKIREYNEALELKDAQ"}'

In [5]:
import requests
requests.get('http://localhost:8000/ptmkb/api/get-protein-details', params={'upac': 'O14746'})

<Response [200]>